In [1]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np

# set some nicer plotting options
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
from pyspark.sql import SQLContext, HiveContext

In [3]:
sqc = SQLContext(sc)
hc = HiveContext(sc)

In [4]:
!hadoop fs -ls /user/roskarr/twitter

Picked up _JAVA_OPTIONS: -Xmx10G -Xms256m -XX:ParallelGCThreads=5
15/11/11 16:56:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 7 items
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:21 /user/roskarr/twitter/2014_05
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:26 /user/roskarr/twitter/2014_06
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:31 /user/roskarr/twitter/2014_07
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:37 /user/roskarr/twitter/2014_09
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:43 /user/roskarr/twitter/2014_10
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:49 /user/roskarr/twitter/2014_11
drwxr-xr-x   - roskarr supergroup          0 2015-11-02 16:50 /user/roskarr/twitter/2014_12


In [5]:
!hadoop fs -du -h /user/roskarr/twitter

Picked up _JAVA_OPTIONS: -Xmx10G -Xms256m -XX:ParallelGCThreads=5
15/11/11 16:56:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
55.1 G  /user/roskarr/twitter/2014_05
31.7 G  /user/roskarr/twitter/2014_06
51.0 G  /user/roskarr/twitter/2014_07
59.6 G  /user/roskarr/twitter/2014_09
61.9 G  /user/roskarr/twitter/2014_10
62.0 G  /user/roskarr/twitter/2014_11
12.1 G  /user/roskarr/twitter/2014_12


In [6]:
%%time
data = hc.read.parquet('/user/roskarr/twitter/2014_10')

CPU times: user 13 ms, sys: 6 ms, total: 19 ms
Wall time: 41.4 s


In [7]:
import pyspark.sql.functions as func
from pyspark.sql import Row
from pyspark.sql import Window
from pyspark.sql.types import IntegerType, ArrayType, StringType, StructField, StructType, DateType, DataType, DateConverter, DatetimeConverter, TimestampType, BooleanType
import datetime

In [8]:
convert_date_string = func.udf(lambda date_string: datetime.date.strftime(datetime.datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'),'%Y-%m-%d %H:%M:%S'), StringType())

In [9]:
# make UDF for converting the date string to a datetime object
datetime_udf = func.udf(lambda date_string: datetime.strptime(date_string, '%a %b %d %H:%M:%S +0000 %Y'), DateType())

In [14]:
hash_text_udf = func.udf(lambda row: [r.text for r in row], returnType=ArrayType(StringType()))

In [15]:
# only keep the tweets with at least one hashtag
hashtag_df = (data.select('created_at', 'entities.hashtags')
                .filter(func.size('hashtags') > 0)
                .withColumn('hash_text', hash_text_udf('hashtags'))
                .select(convert_date_string('created_at').alias('date'), func.explode('hash_text').alias('hashtag'))
                .withColumn('hashtag', func.lower(func.col('hashtag')))
                .repartition(1200))
hashtag_df.cache()

DataFrame[date: string, hashtag: string]

In [16]:
hashtag_df.show(truncate=False)

+-------------------+---------------------------+
|date               |hashtag                    |
+-------------------+---------------------------+
|2014-10-01 06:37:50|芸能                         |
|2014-10-01 18:53:15|miracles                   |
|2014-10-02 00:37:25|horortamanlangsatmayestik  |
|2014-10-02 12:23:46|photography                |
|2014-10-02 17:12:49|falltourfollow             |
|2014-10-02 19:49:59|zi̇raatmühendi̇si̇6500kadro|
|2014-10-03 19:00:54|mgwv                       |
|2014-10-04 05:50:08|gta                        |
|2014-10-04 22:11:40|hadith                     |
|2014-10-05 06:01:07|حقيقه                      |
|2014-10-05 13:00:21|rtagree                    |
|2014-10-05 16:49:22|nsfw                       |
|2014-10-06 17:54:31|iphone                     |
|2014-10-06 23:14:57|vikingstrong               |
|2014-10-07 17:17:07|99inyql                    |
|2014-10-07 21:23:19|amantesdecádiz             |
|2014-10-08 12:16:02|ipadgames                  |


### Simple word count revisited

Now that we have a `DataFrame` consisting of `(timestamp, hashtag)` columns, lets do a simple word count using the `DataFrame` API. This will let us decide later on which hashtags we might be interested in investigating further. 

In [19]:
%%time 
hashtag_df.groupBy('hashtag').count().sort('count', ascending=False).show(50, False)

+--------------------------+-------+
|hashtag                   |count  |
+--------------------------+-------+
|emabiggestfans1d          |1006026|
|emabiggestfansjustinbieber|973621 |
|kcaargentina              |746633 |
|gameinsight               |409308 |
|android                   |300233 |
|تطبيق_قرآنى               |282558 |
|androidgames              |217769 |
|teamfollowback            |186653 |
|rt                        |184199 |
|كنز_المسلم                |173564 |
|رتويت                     |162817 |
|ipad                      |157261 |
|الهلال                    |154979 |
|sougofollow               |154090 |
|相互フォロー                    |143476 |
|rtした人全員フォローする             |140583 |
|ipadgames                 |138735 |
|follow                    |124292 |
|porn                      |123463 |
|الرياض                    |120912 |
|ff                        |118390 |
|السعودية                  |116091 |
|followback                |111813 |
|gonzalohiguain            |109991 |
|

As we can see here, there are lots os languages represented on Twitter! This is very interesting by itself, but for now lets focus on just plain ascii hash tags which we are more likely to understand: 

In [20]:
# define a helper UDF that can be used to filter non-ascii hashtags
def is_ascii(s):
    return all(ord(c) > 0 and ord(c) < 128 for c in s)
is_ascii_udf = func.udf(is_ascii, BooleanType())

In [21]:
hashtag_df = hashtag_df.filter(is_ascii_udf('hashtag'))

### Daily hashtag trends

Now lets do something slightly more complicated and incorporate the time data into our analysis. First, we'll try daily trends and later on do more fine-grained analysis. 

To get information about daily hashtag usage, we must first convert the date string into a number representing day of the year. The [Spark DataFrame functions module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) provides a function [dayofyear](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.dayofyear) that will do this for us and we can use it just like we used the User Defined Functions (UDFs) above. 

In [22]:
daily_hashtag = hashtag_df.select(func.month('date').alias('month'),
                                  func.weekofyear('date').alias('week'),
                                  func.dayofyear('date').alias('day'), 
                                  'hashtag')
daily_hashtag.show(5)

+-----+----+---+--------------------+
|month|week|day|             hashtag|
+-----+----+---+--------------------+
|   10|  40|274|            miracles|
|   10|  40|275|horortamanlangsat...|
|   10|  40|275|         photography|
|   10|  40|275|      falltourfollow|
|   10|  40|276|                mgwv|
+-----+----+---+--------------------+
only showing top 5 rows



We can now use the same technique as above using `groupBy` to get daily counts for each hashtag. Since we want to satisfy two conditions (counts per day and counts per hashtag), we simply give `groupBy` two columns:

In [38]:
day_counts = (daily_hashtag.select('day', 'hashtag', 'week').groupby('day', 'hashtag', 'week')
                       .count())

#### Important aside concerning `count()`

The use of `count()` here is very confusing -- in this case, `count()` is a method of a [GroupedData](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) object, which performs a counting of items in each group. Our grouping here is defined by week, day, and hashtag, so calling `count()` gives us the number of times a hashtag appears each week. To make it even more confusing, `count()` in this case actually returns a `DataFrame`, with "count" as one of the columns. 

However, we are much more used to seeing `count()` as an RDD method! There, it returns the number of elements in the RDD, which is much different. See for example the line below, which uses `count()` as an RDD method and actually just returns the number of rows: 

In [39]:
%time day_counts.sort('day').count()

CPU times: user 144 ms, sys: 58 ms, total: 202 ms
Wall time: 11.6 s


5221650

In [40]:
day_counts.show(5)

+---+-----------+----+-----+
|day|    hashtag|week|count|
+---+-----------+----+-----+
|278|          n|  40|   20|
|281|sougofollow|  41| 4567|
|294|      sehun|  43|  345|
|276|       baby|  40|  112|
|288|         ff|  42| 3813|
+---+-----------+----+-----+
only showing top 5 rows



The same counting of daily occurences of each hashtag can be done with an RDD operation using a `map` and `reduceByKey`, though it might look a bit messier: 

In [35]:
day_counts_rdd = daily_hashtag.rdd.map(lambda r: ((r.day, r.hashtag, r.week), 1)).reduceByKey(lambda a,b: a+b)

In [36]:
%time day_counts_rdd.sortBy(lambda ((day, hashtag, week), count): day).count()

CPU times: user 606 ms, sys: 231 ms, total: 837 ms
Wall time: 33.8 s


5221650

The `DataFrame` API avoids shipping data back and forth to the Python intepreter (and can do some extra optimizations) so the performance difference can be significant, as we can see above. 

Lets make sure that the results are the same by looking up a hashtag in both places:

In [214]:
day_counts_rdd.lookup((274,'retweet',40))

[3238]

In [220]:
day_counts.filter((day_counts.hashtag == 'retweet') & (day_counts.day == 274)).show()

+---+-------+----+-----+
|day|hashtag|week|count|
+---+-------+----+-----+
|274|retweet|  40| 3238|
+---+-------+----+-----+



At this point we should persist `day_counts` in memory since we'll be using it later on. 

In [41]:
day_counts.cache()

DataFrame[day: int, hashtag: string, week: int, count: bigint]

We can get a quick idea of the most popular hashtags by looking at the weekly averages of daily hashtag counts:

In [43]:
day_counts.groupBy('week', 'hashtag').avg('count').sort('avg(count)', ascending=False).show(100)

+----+--------------------+------------------+
|week|             hashtag|        avg(count)|
+----+--------------------+------------------+
|  42|    emabiggestfans1d|54077.857142857145|
|  42|emabiggestfansjus...| 52082.28571428572|
|  44|    emabiggestfans1d|43656.833333333336|
|  44|emabiggestfansjus...|41650.666666666664|
|  43|        kcaargentina| 36302.28571428572|
|  43|    emabiggestfans1d|30190.571428571428|
|  43|emabiggestfansjus...| 29586.85714285714|
|  42|        kcaargentina| 25964.14285714286|
|  40|       votevampsvevo|           21236.0|
|  41|    emabiggestfans1d| 19649.14285714286|
|  41|emabiggestfansjus...| 19479.85714285714|
|  41|        kcaargentina|19261.714285714286|
|  44|        kcaargentina|16576.833333333332|
|  40|        kcaargentina|           15295.0|
|  42|         gameinsight| 14074.42857142857|
|  41|         gameinsight|13817.714285714286|
|  40|         gameinsight|           13668.8|
|  43|         gameinsight| 12464.42857142857|
|  42|       

Apparently, the [MTV Europe Music Awards](https://en.wikipedia.org/wiki/2014_MTV_Europe_Music_Awards) happened in late 2014... (and [yet another boy band](https://en.wikipedia.org/wiki/One_Direction) is on the scene?)

## Using `window` functions

### top daily hashtags

Now lets say we want to know the top tweets for each day. This is a non-trivial thing to try and compute using a standard RDD, but the `DataFrame` API gives us "window" functions that let us do it relatively easily. 

In [51]:
daily_window = Window.partitionBy('day').orderBy(func.desc('count'))

In [52]:
daily_rank = func.rank().over(daily_window)

In [57]:
(day_counts.select('day', 'count', 'hashtag', daily_rank.alias('rank'))
           .filter('rank < 6')
           .show(20))

+---+-----+-------------+----+
|day|count|      hashtag|rank|
+---+-----+-------------+----+
|274| 9880|  gameinsight|   1|
|274| 9179| kcaargentina|   2|
|274| 6675|      android|   3|
|274| 5893| androidgames|   4|
|274| 4618|           rt|   5|
|275|13857| kcaargentina|   1|
|275|13725|  gameinsight|   2|
|275|10093|   1dproposal|   3|
|275| 9774|      android|   4|
|275| 7515| androidgames|   5|
|276|15196|  gameinsight|   1|
|276|14743| kcaargentina|   2|
|276|11497|      android|   3|
|276| 7423| androidgames|   4|
|276| 6806|           ff|   5|
|277|21738|votevampsvevo|   1|
|277|19660| kcaargentina|   2|
|277|15055|  gameinsight|   3|
|277|11135|      android|   4|
|277| 7401| androidgames|   5|
+---+-----+-------------+----+
only showing top 20 rows



### More complicated example: calculating the daily standard deviation based on a weekly window

In [59]:
weekly_window = Window.partitionBy('week', 'hashtag')

In [60]:
mean_diff = day_counts['count'] - func.mean('count').over(weekly_window)

In [61]:
variance_arg = (1./7.)*mean_diff*mean_diff

In [67]:
daily_stats = (day_counts.withColumn('var_arg', variance_arg)
                         .groupBy('week', 'hashtag')
                         .sum('var_arg')
                         .withColumn('stddev', func.sqrt('sum(var_arg)'))
                         .sort('stddev', ascending=False))

In [74]:
daily_stats.show()

+----+--------------------+--------------------+------------------+
|week|             hashtag|        sum(var_arg)|            stddev|
+----+--------------------+--------------------+------------------+
|  41|    emabiggestfans1d| 5.615435298361732E8| 23696.90971068112|
|  41|emabiggestfansjus...| 4.734708501219755E8|21759.385334194885|
|  44|    emabiggestfans1d| 2.938166898330395E8|17141.081932977264|
|  44|        kcaargentina| 2.910696484044708E8|17060.763417985458|
|  44|emabiggestfansjus...|2.5634334961879125E8|16010.726080312263|
|  43|emabiggestfansjus...|1.0385722155091654E8|10191.036333509785|
|  43|    emabiggestfans1d| 9.939805853051284E7| 9969.857498004314|
|  42|    emabiggestfans1d| 6.937371583666532E7| 8329.088535768204|
|  42|emabiggestfansjus...| 5.286668506117162E7|7270.9480166737285|
|  43|stealmygirlvevore...| 4.690633240811635E7|6848.8197821315425|
|  44|  wildlifemusicvideo|2.6747042285687536E7| 5171.754275455045|
|  42|        kcaargentina| 2.605941526528006E7|